## SEC Sentiment Analysis

In [1]:
%load_ext autoreload
%autoreload 2

### Grab 10-K filings from EDGAR

In [2]:
import os

import tqdm

from fetch import get_form_by_ticker

In [3]:
tickers = ['ehc', 'mrk','nke', 'msex', 'v', 'cvs', 'doc', 'smtc', 'cl', 
'ava', 'bc', 'f', 'lmt', 'cri', 'aig', 'rgld', 'apld', 'omcl', 
'mmm', 'bgs', 'dis','wetg', 'bj']

In [4]:
cwd = os.getcwd()
data_directory = os.path.join(cwd, "xbrl-forms")
if not os.path.exists(data_directory):
    os.mkdir(data_directory)

In [5]:
forms = []
for ticker in tqdm.tqdm(tickers): 
    form_text = get_form_by_ticker(
        ticker=ticker,
        form_type="10-K",
        company="Unstructured Technologies",
        email="support@unstructured.io"
    )
    
    filename = os.path.join(data_directory, f"{ticker}-10k.xbrl")
    with open(filename, "w") as f:
        f.write(form_text)

100%|██████████████████████████████████████████████████████████████████████| 23/23 [00:59<00:00,  2.57s/it]


## Extract the Risk Factors Narrative

In [6]:
import requests
import time

In [7]:
url = "https://dev.api.unstructured.io/sec-filings/v0.0.2/section"

In [8]:
risk_factors = dict()
for ticker in tqdm.tqdm(tickers):
    response = requests.post(
        url,
        files={"file": open(f"./xbrl-forms/{ticker}-10k.xbrl", "rb")},
        data={"section": ["RISK_FACTORS"]},
    )
    response.raise_for_status()
    risk_factors[ticker] = response.json()["RISK_FACTORS"]
    time.sleep(1)

100%|██████████████████████████████████████████████████████████████████████| 23/23 [07:48<00:00, 20.36s/it]
